In [48]:
# from noaa_sdk import NOAA
import requests
import pandas as pd 
import numpy as np
import datetime

In [3]:
def get_noaa_data(api_key, dataset_id, station_id, start_date, end_date):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        "datasetid": dataset_id,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "units": "metric",
        "limit": 1000
    }
    headers = {
        "token": api_key
    }
    response = requests.get(base_url, headers=headers, params=params)
    return response.json()

In [79]:
station_id = "GHCND:USC00200228"
fips_id = "26161"
api_key = 'mVuthWQRHnBWzwoEgkijjFUMbRqsNvRh'
dataset_id = "GHCND"
startdate = "2018-01-01"
enddate = "2018-01-31"

In [80]:
data = get_noaa_data(api_key=api_key, 
                     dataset_id=dataset_id,
                     station_id=station_id,
                     start_date=startdate,
                     end_date=enddate
                    )

In [81]:
df = pd.json_normalize(data, 'results')

In [82]:
df.datatype.unique()

array(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'WESD', 'WT01',
       'WT06', 'WT04', 'WT11', 'WT03'], dtype=object)

In [32]:
# url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/datasets'
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND"
params = {
    "id": "GHCND",
    "name": "Daily Summaries",
    "datacoverage": 1,
    "startdate": "2018-01-01",
    "enddate": "2019-01-01"
}
headers = {
    'token': api_key
}
response = requests.get(url, headers=headers, params=params)
data = response.json()
df = pd.json_normalize(data, 'results')
print(df.head())

                  date datatype            station attributes  value
0  2018-01-01T00:00:00     TAVG  GHCND:AE000041196      H,,S,    186
1  2018-01-01T00:00:00     TMAX  GHCND:AE000041196       ,,S,    259
2  2018-01-01T00:00:00     TMIN  GHCND:AE000041196       ,,S,    112
3  2018-01-01T00:00:00     PRCP  GHCND:AEM00041194       ,,S,      0
4  2018-01-01T00:00:00     TAVG  GHCND:AEM00041194      H,,S,    209


In [34]:
data["results"]

[{'date': '2018-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:AE000041196',
  'attributes': 'H,,S,',
  'value': 186},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TMAX',
  'station': 'GHCND:AE000041196',
  'attributes': ',,S,',
  'value': 259},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TMIN',
  'station': 'GHCND:AE000041196',
  'attributes': ',,S,',
  'value': 112},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'PRCP',
  'station': 'GHCND:AEM00041194',
  'attributes': ',,S,',
  'value': 0},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:AEM00041194',
  'attributes': 'H,,S,',
  'value': 209},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TMAX',
  'station': 'GHCND:AEM00041194',
  'attributes': ',,S,',
  'value': 250},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:AEM00041217',
  'attributes': 'H,,S,',
  'value': 191},
 {'date': '2018-01-01T00:00:00',
  'datatype': 'TMIN',
  'station': 'GHCND:AEM00041217',


In [33]:
df.datatype.unique()

array(['TAVG', 'TMAX', 'TMIN', 'PRCP'], dtype=object)

In [27]:
dtypes = requests.get("https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes", 
                      headers=headers, 
                      params = {"id": "GHCND", "name": "Daily Summaries"}).json()

In [28]:
df_dtypes = pd.json_normalize(dtypes, 'results')

In [25]:
df_dtypes.to_csv("../data/dtypes.csv" )

## Aviation Weather Center

In [5]:
endpoint = "https://aviationweather.gov/api/data/metar"
stations = ["PANC"]  # and other stations too
params = {"ids": stations, "format": "json", "hours": 48}
response = requests.get(endpoint, params=params)
df = pd.DataFrame.from_records(response.json())
df["reportTime"] = pd.to_datetime(df["reportTime"])

In [8]:
df.columns

Index(['metar_id', 'icaoId', 'receiptTime', 'obsTime', 'reportTime', 'temp',
       'dewp', 'wdir', 'wspd', 'wgst', 'visib', 'altim', 'slp', 'qcField',
       'wxString', 'presTend', 'maxT', 'minT', 'maxT24', 'minT24', 'precip',
       'pcp3hr', 'pcp6hr', 'pcp24hr', 'snow', 'vertVis', 'metarType', 'rawOb',
       'mostRecent', 'lat', 'lon', 'elev', 'prior', 'name', 'clouds'],
      dtype='object')

In [ ]:
df = df.loc[
    (df["reportTime"].dt.date == datetime.date(2023, 11, 19))  # or whatever the date is
    & (df["metarType"] == "METAR")
][["temp", "precip", "snow"]].agg(["mean", "max", "min", "count"])

y_temp = df["temp"][:3]
y_snow = df["snow"]["max"] > 0
y_rain = df["precip"]["max"] > 0

## Using Meteostat api 

In [27]:
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly

In [6]:
start = datetime(2018, 1, 1)
end = datetime(2018, 2, 15)

In [7]:
from meteostat import Stations

In [8]:
stations = Stations()

In [ ]:
start= datetime.datetime(2022, 11, 12)
end = datetime.datetime(2023, 11, 12)

In [20]:
stations = stations.nearby(42.2229276,-83.7457088)

In [54]:
lat_lons = [(42.2229276,-83.7457088)]

In [62]:
 lat_lons = {
    "KARB" : (42.2231,-83.7453), # Ann Arbor Airport
    "PANC" : (61.1743,-149.9963), # Ted Stevens Anchorage International Airport
    "KBOI" : (43.5644,-116.2228), # Boise Air Terminal
    "KORD" : (41.9742,-87.9073), # Chicago O'Hare International Airport
    "KDEN" : (39.8561,-104.6737), # Denver International Airport
    "KDTW" : (42.2125,-83.3533), # Detroit Metropolitan Airport
    "PHNL" : (21.3187,-157.9225), # Honolulu International Airport
    "KIAH" : (29.9844,-95.3414), # George Bush Intercontinental Airport
    "KMIA" : (25.7933,-80.2906), # Miami International Airport
    "KMIC" : (45.0628,-93.3533), # Minneapolis Crystal Airport
    "KOKC" : (35.3931,-97.6008), # Will Rogers World Airport
    "KBNA" : (36.1244,-86.6782), # Nashville International Airport
    "KJFK" : (40.6397,-73.7789), # John F. Kennedy International Airport
    "KPHX" : (33.4342,-112.0117), # Phoenix Sky Harbor International Airport
    "KPWM" : (43.6461,-70.3092), # Portland International Jetport
    "KPDX" : (45.5886,-122.5975), # Portland International Airport
    "KSLC" : (40.7884,-111.9778), # Salt Lake City International Airport
    "KSAN" : (32.7336,-117.1897), # San Diego International Airport
    "KSFO" : (37.6189,-122.3750), # San Francisco International Airport
    "KSEA" : (47.4489,-122.3094), # Seattle Tacoma International Airport
    "KDCA" : (38.8522,-77.0378), # Ronald Reagan Washington National Airport
}

In [26]:
stations.fetch(1)

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
KARB0,Ann Arbor / Pittsfield,US,MI,<NA>,KARB,42.223,-83.7456,256.0,America/Detroit,2006-01-01,2023-11-17,2006-01-01,2022-04-24,2006-01-01,2022-01-01,12.044687


In [34]:
stations.fetch(1).index[0]

'KARB0'

In [52]:
hourly  = Hourly(stations.fetch(1).index[0], start, end)

In [53]:
(hourly.fetch().isna()*1).mean()

temp    0.0
dwpt    0.0
rhum    0.0
prcp    0.0
snow    1.0
wdir    0.0
wspd    0.0
wpgt    1.0
pres    0.0
tsun    1.0
coco    0.0
dtype: float64

In [66]:
lat_lons.items()

dict_items([('KARB', (42.2231, -83.7453)), ('PANC', (61.1743, -149.9963)), ('KBOI', (43.5644, -116.2228)), ('KORD', (41.9742, -87.9073)), ('KDEN', (39.8561, -104.6737)), ('KDTW', (42.2125, -83.3533)), ('PHNL', (21.3187, -157.9225)), ('KIAH', (29.9844, -95.3414)), ('KMIA', (25.7933, -80.2906)), ('KMIC', (45.0628, -93.3533)), ('KOKC', (35.3931, -97.6008)), ('KBNA', (36.1244, -86.6782)), ('KJFK', (40.6397, -73.7789)), ('KPHX', (33.4342, -112.0117)), ('KPWM', (43.6461, -70.3092)), ('KPDX', (45.5886, -122.5975)), ('KSLC', (40.7884, -111.9778)), ('KSAN', (32.7336, -117.1897)), ('KSFO', (37.6189, -122.375)), ('KSEA', (47.4489, -122.3094)), ('KDCA', (38.8522, -77.0378))])

In [69]:
station_ids = []
for icao, coords in lat_lons.items():
    lat, lon = coords
    station = stations.nearby(*coords).fetch(1)
    station_ids.append(station.index[0])

In [76]:
hourly_df = Hourly(loc = station_ids, start=start, end=end)

In [78]:
hourly_df.fetch().to_csv("../data/hourly_data.csv")